In [0]:
%sql
DROP TABLE IF EXISTS bronze_census_population;


com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:139)
	at scala.Option.getOrElse(Option.scala:201)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:139)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:136)
	at scala.collection.immutable.Range.foreach(Range.scala:192)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:136)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:721)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:441)
	at scala.Option.getOrElse(Option.scala:201)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:441)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark.conf.set("spark.sql.shuffle.partitions", "8")

# --------------------------------------------------
# CORRECT schema – MUST match CSV column order & type
# --------------------------------------------------
census_schema = StructType([
    StructField("region", StringType(), True),
    StructField("year", IntegerType(), True),
    StructField("age_group", StringType(), True),
    StructField("gender", StringType(), True),
    StructField("ethnic_group", StringType(), True),
    StructField("population", IntegerType(), True),
    StructField("literacy_rate", DoubleType(), True),
    StructField("employed", IntegerType(), True),
    StructField("employment_rate", DoubleType(), True)
])

# --------------------------------------------------
# Read CSV from Databricks Volume
# --------------------------------------------------
raw_df = (
    spark.read
    .option("header", True)
    .schema(census_schema)
    .csv("/Volumes/workspace/default/census_volume/census_sample_5000_records.csv")
)

# --------------------------------------------------
# Add audit columns (enterprise standard)
# --------------------------------------------------
bronze_df = (
    raw_df
    .withColumn("ingestion_timestamp", current_timestamp())
    .withColumn("source_path", lit("/Volumes/workspace/default/census_volume"))
)

# --------------------------------------------------
# Write Bronze Delta table
# --------------------------------------------------
bronze_df.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("bronze_census_population")


In [0]:
spark.read.table("bronze_census_population").printSchema()
spark.read.table("bronze_census_population").show(5)


root
 |-- region: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- age_group: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- ethnic_group: string (nullable = true)
 |-- population: integer (nullable = true)
 |-- literacy_rate: double (nullable = true)
 |-- employed: integer (nullable = true)
 |-- employment_rate: double (nullable = true)
 |-- ingestion_timestamp: timestamp (nullable = true)
 |-- source_path: string (nullable = true)

+-------+----+---------+------+------------+----------+-------------+--------+---------------+--------------------+--------------------+
| region|year|age_group|gender|ethnic_group|population|literacy_rate|employed|employment_rate| ingestion_timestamp|         source_path|
+-------+----+---------+------+------------+----------+-------------+--------+---------------+--------------------+--------------------+
|   West|2023|    15-24|  Male|     Group A|     23291|        57.13|    5684|           24.4|2026-01-02 08:06:.